# Taaltheorie en Taalverwerking · 2019 · Week 20

In this assignment, we will work more with WordNet and explore music processing with NLTK tools. Don't forget to load WordNet!

In [65]:
# FILL THIS IN FOR YOUR GROUP, also name your file as: tttv-w20-<group>-<name1>-<name2>.ipynb

# Group        : D
# Name - UvaID : Joshua de Roos - 11242736
# Name - UvaID : Lodewijk van Keizerswaard - 11054115
# Date         : 24-05-2019

In [66]:
from math import *
import nltk
from nltk.corpus import wordnet as wn
# nltk.download('wordnet')

You will also need to install and use the [**requests**](https://2.python-requests.org/en/master/) library for this assignment, either by typing `pip install requests` in a terminal or by using your Python package manager.

In [67]:
import requests

### Question 1 (11 pts total)

In this exercise you will explore the output of a distributional semantic model and compare its semantic similarity ranking to that obtained with path-length distance in an ontology. 

[Indra](http://lambda3.org/Indra/) is a library for working with several distributional semantic models and includes a number of pre-trained models that can be queried online. For a target word, Indra will return its $n$ nearest semantic neighbours ordered by similarity strength, calculated using the similarity measure of your choice.

For this assignment, we will use the Word2Vec model on English-language corpora (`wiki-2018` and `googlenews`), and we will restrict ourselves to the five most similar words according to cosine similarity. We can make a query over `wiki-2018` for the word *sailboat* as follows. We will receive a dictionary containing the most similar words according to Word2Vec, as well as their cosine similarity to *sailboat*. 

In [68]:
r = requests.post('http://indra.lambda3.org/neighbors/relatedness', json = {
        'corpus': 'wiki-2018',
        'model': 'W2V',
        'language': 'EN', 
        'topk': 5, 
        'scoreFunction': 'COSINE',
        'terms': ['sailboat']
})
r.json()

# {'corpus': 'wiki-2018',
#  'model': 'W2V',
#  'language': 'EN',
#  'topk': 5,
#  'terms': {'sailboat': {'sailboat': 0.9999999999999999,
#    'catamaran': 0.7580487287345087,
#    'trimaran': 0.7260682723499513,
#    'dinghy': 0.7202068755210131,
#    'multihull': 0.6924827761338878}}}

{'corpus': 'wiki-2018',
 'language': 'EN',
 'model': 'W2V',
 'terms': {'sailboat': {'catamaran': 0.7580487287345087,
   'dinghy': 0.7202068755210131,
   'multihull': 0.6924827761338878,
   'sailboat': 0.9999999999999999,
   'trimaran': 0.7260682723499513}},
 'topk': 5}

#### Question 1.1 (3 pts)

Search for the target word *potato* using the same parameters as the example above. As output, you should get a list of five words including *potato*, with a similarity score that indicates how similar they are to *potato* according to the model. 

Use NLTK to determine the lowest common hypernym of these five words in *WordNet*.

**Hint:** Don't forget that you need to check all possible senses for each word. The lowest common hypernym is the lowest-level synset that subsumes at least one sense of all five words. You probably *don't* want to use the NLTK **lowest_common_hypernyms()** method. (Can you see why?)

In [69]:
import numpy as np

def get_trees(term):
    synset = wn.synsets(term, pos=wn.NOUN)
    
    trees = []
    for s in synset:
        trees.append(s.hypernym_paths())
        
    nodes = []
    
    for sence in trees:
        for tree in sence:
            for node in tree:
                nodes.append(node)
        
    return set(nodes)

def get_common_hypernym(intersection):
    depth = -1
    ret_node = None
    for node in intersection:
        d = node.min_depth()
        if d > depth:
            depth = d
            ret_node = node
    return ret_node

def get_senses(corpus, word):
    r = requests.post('http://indra.lambda3.org/neighbors/relatedness', json = {
        'corpus': corpus,
        'model': 'W2V',
        'language': 'EN', 
        'topk': 5, 
        'scoreFunction': 'COSINE',
        'terms': [word]
    })
    ret = r.json()
    print(ret)
    ret = list(ret['terms'][word].keys())
    return ret
    

def get_intersection(words):
    intersection = get_trees(words[0])
    for word in words:
        intersection = intersection & get_trees(word)
        
    return intersection
    
    
    
words = get_senses('wiki-2018', 'potato')
intersection = get_intersection(words)
print(get_common_hypernym(intersection))

{'corpus': 'wiki-2018', 'model': 'W2V', 'language': 'EN', 'topk': 5, 'terms': {'potato': {'potato': 1.0, 'potatoes': 0.7151537659675328, 'corn': 0.625528186820859, 'rutabaga': 0.6213762952003541, 'puree': 0.5973323672359723}}}
Synset('matter.n.03')


#### Question 1.2 (3 pts)

Repeat the same exercise using the `googlenews` corpus instead.

In [70]:
words = get_senses('googlenews', 'potato')
print(words)
intersection = get_intersection(words)
print(get_common_hypernym(intersection))

{'corpus': 'googlenews', 'model': 'W2V', 'language': 'EN', 'topk': 5, 'terms': {'potato': {'potato': 1.0, 'potatoes': 0.792025619614087, 'sweet_potato': 0.6572370403424154, 'lentil': 0.6295645627058933, 'asparagus': 0.6216287020728413}}}
['potato', 'potatoes', 'sweet_potato', 'lentil', 'asparagus']
Synset('vascular_plant.n.01')


#### Question 1.3 (3 pts)

Implement a function called **co\_hyponym(node1, node2)** to check whether two *words* (not synsets) are co-hyponyms or sister terms (i.e., whether any sense of one of the words has some immediate hypernym in common with some sense of the other word). 

Which of the semantic neighbours on your lists from the previous two questions are co-hyponyms of *potato*? (Do not count *potatoes*, which has the same lemma as *potato*.)

In [71]:
# Check if they have the same parent.
def co_hyponym(word1, word2):
    sn1 = wn.synsets(word1, pos=wn.NOUN)
    sn2 = wn.synsets(word2, pos=wn.NOUN)
    
    for sense1 in sn1:
        for sense2 in sn2:
            hypernyms = sense1.lowest_common_hypernyms(sense2)
            
            for hypernym in hypernyms:
                if hypernym.shortest_path_distance(sense1) == 1:
                    if hypernym.shortest_path_distance(sense2) == 1:
                        return True
    return False
                

# Make a print statement like the following for every neighbour.
print(co_hyponym("potato", "corn"))
print(co_hyponym("potato", "puree"))
print(co_hyponym("potato", "rutabaga"))
print(co_hyponym("potato", "lentil"))
print(co_hyponym("potato", "sweet_potato"))
print(co_hyponym("potato", "asparagus"))

False
False
False
False
True
False


**Answer**

Only 'sweet_potato' is a co_hyponoym of 'potato'

#### Question 1.4 (2 pts)

Give the results of **textbook_similarity** queries between *potato* and each of the other semantic neighbours from previous questions, using your function from last week. Give the resulting list of semantic neighbours ordered by similarity strength (according to WordNet path-length) and compare this ranking to the rankings from Indra.

In [78]:
import numpy as np
def textbook_similarity(word1, word2, pos):
    if word1 == word2:
        return inf
    
    set1 = wn.synsets(word1, pos=pos)
    set2 = wn.synsets(word2, pos=pos)
    
    if set1 == set2:
        return inf
    
    min_len= inf
    
    for sense1 in set1:
        for sense2 in set2:
            if sense1.root_hypernyms() != sense2.root_hypernyms():
                continue
                
            path_len = sense1.shortest_path_distance(sense2)
            if path_len < min_len:
                min_len = path_len
            
    if min_len == inf:
        return -min_len
    return -log(min_len)

# Make a print statement like the following for every neighbour.
neighbours = ["potatoes", "potato", "corn", "puree", "rutabaga", "lentil", "sweet_potato", "asparagus"]
similarity = [(n, float(textbook_similarity("potato", n, pos=wn.NOUN))) for n in neighbours]
similarity = sorted(similarity, key = lambda x: x[1])
similarity.reverse()
print([n[0] for n in similarity])

['potato', 'potatoes', 'sweet_potato', 'asparagus', 'rutabaga', 'lentil', 'corn', 'puree']


**Answer**
The ranking according to textbook_similarity is a little different than the Indra-rankings. In our ranking the order of most similar neigbours is 'rutabaga'-'corn'-'puree' whereas in the Indra ranking the order is 'corn'-'rutabaga'-'puree'


## Question 2: Parsing Eurovision (7 pts total)

This question will introduce the basics of grammar-based syntactic analysis of musical harmony.

### Question 2.1 (1 pt)

Go to http://chordify.net and run an analysis of France's official entry for the Eurovision Song Contest 2019 by copying the following link to the search: https://www.youtube.com/watch?v=dw7WqoSHtgU. Ignore the Eurovision branding at the beginning of the video. Starting from the entrance of the piano at the beginning of the song, copy the first 7 chords as they appear in the Chordify interface, as a list of Python strings. You may find it slightly faster if you change to the *Akkorden* view instead of *Diagrammen*. Use the '#' character for sharp signs.

**Answer:**

In [79]:
french_chords = ['F#m', 'E', 'D', 'F#', 'C#', 'F#m', 'E']

### Question 2.2 (3 pts)

The following grammar is a (somewhat simplified) implementation of the harmonic grammar Fred Lerdahl proposes in his book *Tonal Pitch Space* (2001). In addition to the traditional harmonic classes of tonic, dominant, and subdominant harmony, Lerdahl's grammar includes a *departure* class (`Dep`), a *return* class (`Ret`), and a *neighbour* class (`N`). His definitions of the harmonic classes in terms of Roman numerals are commented out: the 'lexical rules' for this grammar.

The tonal centre of France's Eurovision entry is F$\sharp$ (Chordify gets it wrong). Use the principles we discussed in class to replace the Roman-numeral lexical rules with lexical rules for all of the chord symbols in `french_chords` (e.g., `T -> 'G'` for pieces in the key of G).

Run the code block when you are finished to see how many parse trees Lerdahl's grammar can compute for this sentence.

**Hint:** If you still find the rules for converting between chord names and Roman numerals confusing, try browsing Wikipedia's surprisingly good collection of articles on harmony, starting here: https://en.wikipedia.org/wiki/Roman_numeral_analysis

In [80]:
from nltk import CFG
from nltk.parse.chart import LeftCornerChartParser

lerdahl_grammar = CFG.fromstring("""
  P -> T
  T -> T T
  T -> D T
  D -> D D
  D -> S D
  S -> S S
  T -> T N T
  D -> D N D
  S -> S N S
  T -> T Dep
  D -> D Dep
  S -> S Dep
  Dep -> N Dep
  T -> Ret T
  D -> Ret D
  S -> Ret S
  
  T -> 'F#m' | 'F#'
  D -> 'C#' | 'E'
  S -> 'D'
  Dep -> 'C#' | 'E' | 'D'
  Ret -> 'C#' | 'E' | 'D' 
  N -> 'C#' | 'E' | 'D'
""")

lerdahl_parser = LeftCornerChartParser(lerdahl_grammar)
lerdahl_parses = lerdahl_parser.parse(french_chords)

# Print the total number of parses as well as the actual parse trees.
lerdahl_sum = 0
for t in lerdahl_parses: 
    lerdahl_sum = lerdahl_sum + 1
    t.pretty_print()
print(lerdahl_sum, 'trees')

                 P                     
                 |                      
                 T                     
          _______|___________           
         |                   T         
         |                ___|_______   
         T               T           | 
      ___|___         ___|_______    |  
     T       |       T           |   | 
  ___|___    |    ___|___        |   |  
 T      Dep Dep  T      Dep      T  Dep
 |       |   |   |       |       |   |  
F#m      E   D   F#      C#     F#m  E 

                 P                 
                 |                  
                 T                 
          _______|_______           
         |               T         
         |            ___|_______   
         T           T           | 
      ___|___     ___|___        |  
     T       |   |       T       | 
  ___|___    |   |    ___|___    |  
 T      Dep Dep  T   D       T  Dep
 |       |   |   |   |       |   |  
F#m      E   D   F#  C#     F#m  E 


             P                 
             |                  
             T                 
          ___|_______________   
         T                   | 
  _______|___                |  
 |           T               | 
 |    _______|___            |  
 |   |           T           | 
 |   |    _______|___        |  
 |   |   |           T       | 
 |   |   |        ___|___    |  
 |   |   |       T       |   | 
 |   |   |    ___|___    |   |  
 T  Ret Ret  T      Dep  T  Dep
 |   |   |   |       |   |   |  
F#m  E   D   F#      C# F#m  E 

             P                     
             |                      
             T                     
          ___|___________________   
         T                       | 
  _______|___                    |  
 |           T                   | 
 |    _______|___                |  
 |   |           T               | 
 |   |    _______|___            |  
 |   |   |           T           | 
 |   |   |    _______|___        |  
 |   |  

### Question 2.3 (2 pts)

The next grammar is based on the foundations of Chordify (Bas de Haas, *Music Information Retrieval Based on Tonal Harmony*, 2012). Chordify uses a different set of harmonic classes: tonic, dominant, subdominant, and *tonic prolongation* (`TPG`). They also define the members of the traditional classes differently than Fred Lerdahl; theirs  are based on a beautiful branch of music theory known as neo-Riemannian theory. Replace the commented 'lexical rules' in this grammar with actual chord symbols from `french_chords`, just like you did in the previous question, and run the block.

In [81]:
chordify_grammar = CFG.fromstring("""
  P -> PCPa
  P -> PCPb
  P -> HCP
  P -> P P

  PCPa -> D T | D D T
  PCPa -> PCPa T
  PCPb -> T D T
  HCP -> T D
  HCP -> T HCP
  D -> S D
  T -> TPG
  
  T -> T T
  D -> D D
  S -> S S
  
  T -> 'F#m' | 'F#'
  D -> 'C#' | 'E'
  TPG -> 'D'
""")

chordify_parser = LeftCornerChartParser(chordify_grammar)
chordify_parses = chordify_parser.parse(french_chords)

# Print the total number of parses as well as the actual parse trees.
chordify_sum = 0
for t in chordify_parses: 
    chordify_sum = chordify_sum + 1
    t.pretty_print()
print(chordify_sum, 'trees')

                     P                 
              _______|___________       
             P                   |     
      _______|_______            |      
     |               P           |     
     |               |           |      
     |              HCP          |     
     |            ___|___        |      
     P           T       |       P     
     |        ___|___    |       |      
    HCP      T       |   |      HCP    
  ___|___    |       |   |    ___|___   
 T       D  TPG      T   D   T       D 
 |       |   |       |   |   |       |  
F#m      E   D       F#  C# F#m      E 

                 P                     
              ___|_______________       
             P                   |     
      _______|___                |      
     |           P               |     
     |           |               |      
     P          HCP              P     
     |        ___|___            |      
    HCP      T      HCP         HCP    
  ___|___    |    ___|___   

### Question 2.4 (1 pt)

Which grammar is more practical for daily use, Lerdahl's or Chordify's? Why?

**Answer:**
Chordify's grammar is more practical since it only produces 6 trees instead of 385 as Lerdahl's grammar does. This means that Chordify will be much faster in use, which is better for daily use.